In [ ]:
# Import Libraries
import pandas as pd

In [ ]:
def alpha_diversity_plot():
    """function that loads and cleans alpha diversity metrics into dataframe for use in dash boxplot"""
    # Loading and cleaning alpha diversity metrics
    shannon = pd.read_csv('shannon-alpha-diversity.tsv', sep='\t')
    shannon.columns = ['sample-id','shannon_entropy']

    faith = pd.read_csv('faith-alpha-diversity.tsv', sep='\t')
    faith.columns = ['sample-id','faith_diversity']

    pielou = pd.read_csv('pielou-alpha-diversity.tsv', sep='\t')
    pielou.columns = ['sample-id','pielou_evenness']

    observed = pd.read_csv('observed-features-alpha-diversity.tsv', sep='\t')
    observed.columns = ['sample-id','observed_features']
    
    # Load metadata from QIIME2 and append diversity metrics
    alpha_metadata = pd.read_csv('metadata.tsv', sep='\t')
    alpha_metadata = alpha_metadata.drop(alpha_metadata.index[0])
    alpha_metadata = alpha_metadata.reset_index(drop=True)
    alpha_metadata = alpha_metadata.merge(shannon,how='outer')
    alpha_metadata = alpha_metadata.merge(faith,how='outer')
    alpha_metadata = alpha_metadata.merge(pielou,how='outer')
    alpha_metadata = alpha_metadata.merge(observed,how='outer')
    return alpha_metadata

In [ ]:
def test_alpha_diversity_plot():
    
    return

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

df = alpha_diversity_plot()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("x-axis:"),
    dcc.RadioItems(
        id='x-axis', 
        options=[{'value': x, 'label': x}  
                 for x in df.columns],
        value='subject', 
        labelStyle={'display': 'inline-block'}
    ),
    html.P("y-axis:"),
    dcc.RadioItems(
        id='y-axis', 
        options=[{'value': x, 'label': x} 
                 for x in ['shannon_entropy','faith_diversity','pielou_evenness','observed_features']],
        value='shannon_entropy', 
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="box-plot"),
])

@app.callback(
    Output("box-plot", "figure"), 
    [Input("x-axis", "value"), 
     Input("y-axis", "value")])
def generate_chart(x, y):
    fig = px.box(df, x=x, y=y)
    return fig

app.run_server(debug=True, use_reloader=False)